# Pr-Intern Tests

In [13]:
from huggingface_hub import login
import matplotlib.pyplot as plt
import torch

from geometry_experiments.test_utils import *
login(
    token="..."
)

## Experiment Configurations
Define different experimental setups with various model configurations

In [3]:
experiments={
    'exp2':{'language':False,'vision':False,'expert':False,'lora':False,'path':"meta-llama/Llama-3.2-11B-Vision-Instruct"},
}

## Model loader

In [4]:
def load_and_prepare_model(experiment):
    
    model, processor, vision_expert = load_model_and_processor(
        experiment,
    )

    model.eval()
    return model, processor, vision_expert

model, processor, vision_expert = load_and_prepare_model(experiments['exp2'])


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Prompt preparation

In [16]:
def high_level_planning(request):
    with open("/home/cm2161/Documents/llama-manufacturing/pr-intern/geometry_experiments/0_high_level_planning.txt", "r") as file:
        prompt_template = file.read()
    return prompt_template.replace("{request}", f"{request}")

def individual_ideation(request):
    with open("/home/cm2161/Documents/llama-manufacturing/pr-intern/geometry_experiments/1_individual_ideation.txt", "r") as file:
        prompt_template = file.read()
    return prompt_template.replace("{request}", f"{request}")

def low_level_planning(request):
    with open("/home/cm2161/Documents/llama-manufacturing/pr-intern/geometry_experiments/2_low_level_planning.txt", "r") as file:
        prompt_template = file.read()
    return prompt_template.replace("{request}", f"{request}")

def gcode_generation(request):
    with open("/home/cm2161/Documents/llama-manufacturing/pr-intern/geometry_experiments/3_gcode_generation.txt", "r") as file:
        prompt_template = file.read()
    return prompt_template.replace("{request}", f"{request}")

request="A single layer circle with radius 10mm, and 100%  \infill density with concentric pattern."
steps=[individual_ideation(request)]


## Answer Generation

In [17]:
for i,step in enumerate(steps):

    if i>0:
        step = step.replace("{last_answer}", last_answer)

    with torch.no_grad():
        batch_collated = val_collate_fn_text(step, processor).to(model.device)

        # Generate outputs
        output = model.generate(
            **batch_collated,
            max_new_tokens=10000,
            temperature=0.1
        )

        # Process results
        decoded_output = map(processor.decode, output)
        answer = answer_extractor(decoded_output)
        last_answer= answer[0]
        print(last_answer)


Based on the provided high-level plan, I will break down the task into simpler tasks and create a Python function for each shape. We will start with the circle.

### Circle Shape

To create a circle in 3D printing, we will use the `G2` and `G3` G-code commands for arc movements. We will also use the `G1` command for linear movements.

#### Circle G-code Generation Function

```python
def generate_circle_gcode(circle_properties):
    """
    Generate G-code for a circle shape.

    Args:
        circle_properties (dict): Dictionary containing circle properties.
            - type (str): Type of shape (always 'circle' for this function).
            - geometric_properties (dict): Geometric properties of the circle.
                - radius (float): Radius of the circle in mm.
                - height (float): Height of the circle in mm (single layer).
                - start_position (list): [x, y, z] coordinates of the start position.
                - rotation (list): [x, y, z] rotatio

In [ ]:
def generate_circle_gcode(circle_properties):
    """
    Generate G-code for a circle shape.

    Args:
        circle_properties (dict): Dictionary containing circle properties.
            - type (str): Type of shape (always 'circle' for this function).
            - geometric_properties (dict): Geometric properties of the circle.
                - radius (float): Radius of the circle in mm.
                - height (float): Height of the circle in mm (single layer).
                - start_position (list): [x, y, z] coordinates of the start position.
                - rotation (list): [x, y, z] rotation of the circle.
            - infill (str): Infill pattern (always 'concentric' for this function).
            - infill_density (str): Infill density (always '100%' for this function).
            - layer_height (float): Layer height in mm.
            - layers (int): Number of layers (always 1 for this function).

    Returns:
        str: G-code for the circle shape.
    """

    # Extract circle properties
    radius = circle_properties['geometric_properties']['radius']
    height = circle_properties['geometric_properties']['height']
    start_position = circle_properties['geometric_properties']['start_position']
    rotation = circle_properties['geometric_properties']['rotation']
    layer_height = circle_properties['layer_height']

    # Calculate the number of points for the circle
    num_points = int(2 * 3.14 * radius / layer_height)

    # Initialize G-code string
    gcode = ""

    # Move to the start position
    gcode += f"G1 X{start_position[0]} Y{start_position[1]} Z{start_position[2]}\n"

    # Rotate to the correct orientation
    gcode += f"G21 G92 E0\n"  # Set units to mm and reset extruder position
    gcode += f"G28 X0 Y0\n"  # Home X and Y axes
    gcode += f"G1 Z{start_position[2]}\n"  # Move to Z height
    gcode += f"G1 X{start_position[0]} Y{start_position[1]}\n"  # Move to start position
    gcode += f"G1 Z-{height}\n"  # Move down to the layer height
    gcode += f"G21 G92 E0\n"  # Set units to mm and reset extruder position

    # Generate circle points
    for i in range(num_points):
        angle = 2 * 3.14 * i / num_points
        x = start_position[0] + radius * math.cos(angle)
        y = start_position[1] + radius * math.sin(angle)
        z = start_position[2] - height
        gcode += f"G2 X{x} Y{y} I0 J0\n"

    # Move back to the start position
    gcode += f"G1 X{start_position[0]} Y{start_position[1]}\n"

    return gcode

